## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# change path to reflect your env
%cd /content/gdrive/MyDrive/ml-final/ML-Final-Project
! pip3 install -r requirements.txt

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/ml-final/ML-Final-Project/src')
sys.path.append('/content/drive/MyDrive/ml-final/ML-Final-Project/src/utils')

%cd /content/gdrive/MyDrive/ml-final/ML-Final-Project/src

In [ ]:
#!pip install numpy --upgrade
!pip install cebra
#!pip install h5py

In [ ]:
# this cell sometimes needs to be run again after the following cell in order to correctly load the dataset
# we've tried to fix it but it remains like this -- not sure why!

import torch

import matplotlib.pyplot as plt
import numpy as np
import joblib as jl
import cebra.datasets
from cebra import CEBRA
from utils.nets import *
from utils.model_tools import *
import utils.ciivae_etc as UTIL
from scipy.ndimage import gaussian_filter1d

In [ ]:
!pip install 'cebra[datasets,demos]'
#os.environ['CEBRA_DATADIR'] = "./data"

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

seed = 420

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
ls data/rat_hippocampus/

## Import data/Preprocess

In [ ]:
DATA_NAME = 'rat-hippocampus-achilles-3fold-trial-split-0'
offset_right = 5
offset_left = 5

def _call_dataset(offset_right, offset_left, split):
    dataset = cebra.datasets.init(DATA_NAME, split = split)
    dataset.offset.right = offset_right
    dataset.offset.left = offset_left
    return dataset

train_set = _call_dataset(offset_right, offset_left, 'train')
valid_set = _call_dataset(offset_right, offset_left, 'valid')
test_set = _call_dataset(offset_right, offset_left, 'test')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def make_loader(dataset, batch_size):
    tensor_dataset = TensorDataset(dataset[torch.arange(len(dataset))], dataset.index)
    dataloader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=True)
    return tensor_dataset, dataloader

train_dataset, train_loader = make_loader(train_set, 200)
valid_dataset, valid_loader = make_loader(valid_set, 200)
test_dataset, test_loader = make_loader(test_set, 200)

train_features, train_labels = next(iter(train_loader))
print(train_features[0].shape)
print(train_labels[0].shape)

In [ ]:
fig = plt.figure(figsize=(9,3))
plt.subplots_adjust(wspace = 0.3)
ax = plt.subplot(121)
ax.imshow(train_set.neural.numpy()[:1000].T, aspect = 'auto', cmap = 'gray_r')
plt.ylabel('Neuron #')
plt.xlabel('Time [s]')
plt.xticks(np.linspace(0, 1000, 5), np.linspace(0, 0.025*1000, 5, dtype = int))

ax2 = plt.subplot(122)
ax2.scatter(np.arange(1000), train_set.continuous_index[:1000,0], c = 'gray', s=1)
plt.ylabel('Position [m]')
plt.xlabel('Time [s]')
plt.xticks(np.linspace(0, 1000, 5), np.linspace(0, 0.025*1000, 5, dtype = int))
plt.show()

print(train_set.neural.numpy().shape)

## Train CI-iVAE

In [ ]:
result_path='./results/' + DATA_NAME + '/' + str(seed)

ci_ivae = CIiVAE(dim_x=train_set.neural.shape[1], dim_u=3, dim_z=2) #CONV_iVAE.ConvCIiVAE(dim_x=train_set.neural.shape[1], dim_u=3, dim_z=2)

# train_x is a list of batches typically handled by the loader. TODO figure out how to do that in our implementation
# also: they say something about applying the pi-vae preprocessing script to this data
# it's unclear if they actually run the script while loading it or if we should
# but that code is available so if we need to it's not a big deal

dataloader_dict = {}
dataloader_dict['train'] = train_loader
dataloader_dict['val'] = valid_loader
dataloader_dict['test'] = test_loader

# # train CI-iVAE networks. Results will be saved at the result_path
fit(model=ci_ivae, dataloader_dict=dataloader_dict, seed=seed, num_epoch=260, init_lr=5e-4, lr_milestones=[20, 50, 150], result_path=result_path)
#CONV_iVAE.fit(model=ci_ivae, x_train=train_x, u_train=train_u,
#            x_val=valid_x, u_val=valid_u, seed=seed)

## Load CI-iVAE and evaluate

In [ ]:
ci_ivae = CIiVAE(dim_x=train_set.neural.shape[1], dim_u=3, dim_z=2)

state_dict = torch.load(result_path + '/model.pth', map_location=torch.device('cpu'))

ci_ivae[0] = state_dict['prior']
ci_ivae[1] = state_dict['encoder']
ci_ivae[2] = state_dict['decoder']

In [ ]:
def evaluate_ciivae(model, x, u, sigma=9/6, device='cpu'):
    # feed all in at once
    prior, encoder, decoder = model
    
    prior.eval()
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        lam_mean, lam_log_var = prior(u)
        x_reshape = torch.transpose(x, 1, 2)
        fire_rate = torch.squeeze(UTIL.gaussian_conv(x_reshape, sigma, device='cpu'))

        z_mean, z_log_var = encoder(fire_rate)

        z_mean = torch.squeeze(z_mean)
        z_log_var = torch.squeeze(z_log_var)
        post_mean, post_log_var = UTIL.compute_posterior(z_mean, z_log_var, lam_mean, lam_log_var)
    
        return post_mean, z_mean

    
post_mean, z_mean = evaluate_ciivae(ci_ivae, train_set[torch.arange(len(train_set))], train_set.index)
labels = train_set.index.numpy()

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111)
r_ind = labels[:,1] == 1
l_ind = labels[:,2] == 1
ax.scatter(post_mean[r_ind, 0], post_mean[r_ind, 1], c=labels[r_ind,0], s=1, cmap = 'cool')
ax.scatter(post_mean[l_ind, 0], post_mean[l_ind, 1], c=labels[l_ind,0], s=1, cmap = 'viridis')

print(post_mean.shape)

plt.savefig(result_path + '/latents_train.png') 

## Examining Consistency Across Subjects

In [ ]:
# load in all subjects
# fit linear models
# get r2 models

r2 = [[] * 4] * 4
post_mean_all = {}
z_mean_all = {}

hippocampus_pos_train = {}
hippocampus_pos_test = {}
hippocampus_pos_valid = {}

rats = ['buddy', 'cicero', 'gatsby', 'achilles']

for rat in rats:
  data_name = f'rat-hippocampus-{rat}-3fold-trial-split-0'
  offset_right = 5
  offset_left = 5

  result_path = './results/' + data_name

  hippocampus_pos_train[rat] = _call_dataset(offset_right, offset_left, 'train', data_name=data_name)
  #hippocampus_pos_valid[rat] = _call_dataset(offset_right, offset_left, 'valid')
  #hippocampus_pos_test[rat] = _call_dataset(offset_right, offset_left, 'test')

  train_dataset, train_loader = make_loader(hippocampus_pos_train[rat], 200)
  #valid_dataset, valid_loader = make_loader(valid_set, 200)
  #test_dataset, test_loader = make_loader(test_set, 200)

  ci_ivae = CIiVAE(dim_x=hippocampus_pos_train[rat].neural.shape[1], dim_u=3, dim_z=2)

  state_dict = torch.load(result_path + '/model.pth', 
                          map_location=torch.device('cpu'))


  ci_ivae[0] = state_dict['prior']
  ci_ivae[1] = state_dict['encoder']
  ci_ivae[2] = state_dict['decoder']
  post_mean_train, z_mean_train = evaluate_ciivae(ci_ivae,
                                      hippocampus_pos_train[rat][torch.arange(len(hippocampus_pos_train[rat]))],
                                      hippocampus_pos_train[rat].index)
  #post_mean_test, z_mean_train = evaluate_ciivae(ci_ivae,
  #                                    train_set[torch.arange(len(train_set))],
  #                                    train_set.index)
  #post_mean_train, z_mean_train = evaluate_ciivae(ci_ivae,
  #                                    train_set[torch.arange(len(train_set))],
  #                                    train_set.index)
  post_mean_all[rat] = post_mean_train.numpy()
  print(post_mean_train.numpy().shape)
  z_mean_all[rat] = z_mean_train

del(ci_ivae)
del(state_dict)

# labels to align the subjects is the position of the mouse in the arena
labels_train = [hippocampus_pos_train[rat].continuous_index[:, 0]
          for rat in list(hippocampus_pos_train.keys())]
#labels_test = [hippocampus_pos_test[rat].continuous_index[:, 0]
#          for rat in list(hippocampus_pos_test.keys())]
#labels_valid = [hippocampus_pos_valid[rat].continuous_index[:, 0]
#          for rat in list(hippocampus_pos_valid.keys())]

# consistencies
time_scores_train, time_pairs_train, time_subjects_train = cebra.sklearn.metrics.consistency_score(embeddings=list(post_mean_all.values()),
                                                                                 labels=labels_train,
                                                                                 dataset_ids=rats,
                                                                                 between="datasets")


fig = plt.figure(figsize=(11, 4))

ax1 = plt.subplot(121)

ax1 = cebra.plot_consistency(time_scores_train, pairs=time_pairs_train, datasets=time_subjects_train,
                             ax=ax1, title="Conv-CI-iVAE", colorbar_label=None, vmin=0, vmax=100)
